In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
from PIL import Image
import os
import pathlib

In [2]:
image_path = "images/pic1.jpeg"

In [3]:
def cv2pil(image):
    ''' OpenCV型 -> PIL型 '''
    new_image = image.copy()
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
    elif new_image.shape[2] == 4:  # 透過
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGRA2RGBA)
    new_image = Image.fromarray(new_image)
    return new_image

In [4]:
def mask_image(image_path, face = "face_detector", model = "mask_detector.model", confidence = 0.5):
    
    parms = {}
    parms["image"] = image_path
    parms["face"] = face
    parms["model"] = model
    parms["confidence"] = confidence

    # load our serialized face detector model from disk
    print("[INFO] loading face detector model...")
    prototxtPath = os.path.sep.join([parms["face"], "deploy.prototxt"])
    weightsPath = os.path.sep.join([parms["face"],
                                    "res10_300x300_ssd_iter_140000.caffemodel"])
    net = cv2.dnn.readNet(prototxtPath, weightsPath)

    # load the face mask detector model from disk
    print("[INFO] loading face mask detector model...")
    model = load_model(parms["model"])

    # load the input image from disk, clone it, and grab the image spatial
    # dimensions
    image = cv2.imread(parms["image"])
    orig = image.copy()
    (h, w) = image.shape[:2]

    # construct a blob from the image
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
                                 (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    print("[INFO] computing face detections...")
    net.setInput(blob)
    detections = net.forward()

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the detection
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if confidence > parms["confidence"]:
            # compute the (x, y)-coordinates of the bounding box for
            # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of
            # the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            face = image[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)

            # pass the face through the model to determine if the face
            # has a mask or not
            (mask, withoutMask) = model.predict(face)[0]

            # determine the class label and color we'll use to draw
            # the bounding box and text
            label = "Mask" if mask > withoutMask else "No Mask"
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

            # include the probability in the label
            label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

            # display the label and bounding box rectangle on the output
            # frame
            cv2.putText(image, label, (startX, startY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
            print(label)

    # show the output image
    display(cv2pil(image))
#     cv2.imwrite('./images/output.png', image)
#     cv2.imshow("Output", image)
#     cv2.waitKey(0)

In [5]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('./images/images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('images/images/1.jpg'),
 PosixPath('images/images/10.jpg'),
 PosixPath('images/images/11.jpg'),
 PosixPath('images/images/12.jpg'),
 PosixPath('images/images/13.jpg'),
 PosixPath('images/images/14.jpg'),
 PosixPath('images/images/15.jpg'),
 PosixPath('images/images/16.jpg'),
 PosixPath('images/images/17.jpg'),
 PosixPath('images/images/18.jpg'),
 PosixPath('images/images/19.jpg'),
 PosixPath('images/images/2.jpg'),
 PosixPath('images/images/20.jpg'),
 PosixPath('images/images/3.jpg'),
 PosixPath('images/images/4.jpg'),
 PosixPath('images/images/5.jpg'),
 PosixPath('images/images/6.jpg'),
 PosixPath('images/images/7.jpg'),
 PosixPath('images/images/8.jpg'),
 PosixPath('images/images/9.jpg')]

In [6]:
str(TEST_IMAGE_PATHS[0])

'images/images/1.jpg'

In [ ]:
for img_path in TEST_IMAGE_PATHS:
    mask_image(str(img_path))